# Recursión

Tanto en informática como en matemáticas es habitual resolver problemas reduciéndolos a instancias más sencillas de los mismos, hasta encontrar ejemplares que sabemos resolver de forma trivial. Este es el fundamento de la inducción en matemáticas: se prueba una propiedad de forma directa para uno o más *casos base* y para el resto de casos (llamados *inductivos*) se deduce asumiendo que la propiedad es cierta para instancias más pequeñas. Por ejemplo, $\sum_{k=1}^n k = n (n + 1) / 2$ porque $0 = 0$ para $n=0$ (caso base) y $\sum_{k=1}^n k = \sum_{k=1}^{n-1} k + n = (n - 1) n / 2 + n = (n^2 - n + 2n) / 2 = n (n + 1) / 2$ (caso inductivo).

En programación, la *recursión* consiste en definir funciones que se llaman a sí mismas sobre instancias más pequeñas del problema que pretenden resolver. Igualmente habrá casos base donde la función se resuelva directamente, mientras que el resto de casos se resolverán combinando los resultados de una o más llamadas recursivas sobre instancias más sencillas del problema.

**Ejemplo 1 (sucesiones):** la recursión se puede aplicar trivialmente al cálculo del término $n$-ésimo de una sucesión definida en función de los términos anteriores. Por ejemplo, el factorial, $x_0 = 1$, $x_n = n \cdot x_{n-1}$.

In [1]:
def factorial(n: int) -> int:
    if n == 0:  # caso base
        return 1
    else:  # caso recursivo
        return n * factorial(n - 1)

In [2]:
factorial(6)

720

*Observación 1*: ¿qué pasa si ejecutamos `factorial(-1)`? Se ejecutará el bloque `else` porque $n \neq 0$ y se llamará a `factorial(-2)`, luego a `factorial(-3)` y así sucesivamente sin fin. Como con los bucles `while`, es posible que una función recursiva no termine si está mal diseñada o si se llama con unos argumentos inesperados. Si hubiéramos definido el caso base como `n <= 0` la ejecución infinita desaparecería, o incluso podríamos comprobar si el número es no negativo y lanzar una excepción en caso contrario. Sin embargo, es razonable que la función garantice un resultado válido solo cuando los parámetros de entrada cumplen una *precondición* (aquí $n \geq 0$) y en caso contrario no asegure absolutamente nada. <bigskip/>

**Ejemplo 1 bis:** la sucesión de Fibonacci, $f_0 = 0$, $f_1 = 1$, $f_n = f_{n-1} + f_{n-2}$.

In [3]:
def fibonacci(n: int) -> int:
    if n == 0 or n == 1:  # caso base
        return n
    else:  # caso inductivo
        return fibonacci(n - 1) + fibonacci(n - 2)

In [4]:
fibonacci(6)

8

Los problemas recursivos se pueden resolver también mediante programas *iterativos*, es decir, con bucles y las instrucciones que se han visto anteriormente. De hecho, estas funciones ya se implementaron con bucles en el primer cuatrimestre.

In [5]:
def fibonacci_iter(n: int) -> int:
    fib_nm1, fib_n = 0, 1
    
    for k in range(2, n + 1):
        fib_nm1, fib_n = fib_n, fib_nm1 + fib_n
        
    return fib_n

In [6]:
%timeit fibonacci(30)

226 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit fibonacci_iter(30)

1.15 µs ± 25.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Los programas recursivos suelen ser más abstractos, simples y comprensibles que los programas iterativos, pero también es habitual que sean menos eficientes. El caso de la sucesión de Fibonacci es particularmente atroz, porque sus dos llamadas recursivas provocan una cantidad exponencial de trabajo duplicado (como probaremos al final del tema).

Generalizando la función `fibonacci` para que devuelva tanto $f_n$ como $f_{n-1}$, se obtiene una implementación recursiva de Fibonacci de una eficiencia comparable a la iterativa.

In [8]:
def fibonacci_mejor(n: int) -> tuple[int, int]:
    if n == 0:  # caso base
        return 0, None
    elif n == 1:  # caso base
        return 1, 0
    else:  # caso inductivo
        fibnm1, fibnm2 = fibonacci_mejor(n - 1)
        # La función recursiva devuelve también el anterior
        return fibnm1 + fibnm2, fibnm1

In [9]:
fibonacci_mejor(6)

(8, 5)

In [10]:
%timeit fibonacci_mejor(30)

3.85 µs ± 90.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


No obstante, la implementación recursiva sigue siendo más lenta, debido al coste de apilar los sucesivos entornos de la pila de llamadas. <bigskip/>

**Ejemplo 2 (palíndromos):** la recursión no solo se puede aplicar a números naturales. Por ejemplo, podemos programar una función recursiva que compruebe si una cadena de texto es un palíndromo. $a_1 a_2 \cdots a_{n-1} a_n$ será palíndromo si $a_1 = a_n$ y $a_2 \cdots a_{n-1}$ es palíndromo. Observa cómo hemos reducido el problema a una instancia más pequeña del mismo. La cadena vacía es también un palíndromo y será el caso base. Esto lo podemos escribir de forma esquemática como
$$\begin{array}{rcl}
    \mathrm{palíndromo}() &=& \mathit{cierto} \\ 
    \mathrm{palíndromo}(a_1 a_2 \cdots a_{n-1} a_n) &=& a_1 = a_n \;\wedge\; \mathrm{palíndromo}(a_2 \cdots a_{n-1})
\end{array}
$$
y en Python como la siguiente función:

In [11]:
def es_palíndromo(texto: str) -> bool: 
    if texto == '':  # caso base
        return True
    else:            # caso inductivo
        return texto[0] == texto[-1] and es_palíndromo(texto[1:-1])

In [12]:
es_palíndromo('dabale arroz a la zorra el abad'.replace(' ', ''))

True

In [13]:
es_palíndromo('palindromo')

False

**Ejemplo 3 (búsqueda binaria):** en el primer cuatrimestre se vio el método de la bisección para encontrar soluciones aproximadas a problemas numéricos. Esa idea también se puede aplicar a la búsqueda eficiente dentro de una lista ordenada.

In [14]:
def busca_binaria(v: list, elem) -> tuple[bool, int]:
    """Busca elem en v"""
    
    # Lista vacía
    if not v:
        return False, 0
    
    # Comprueba el elemento en la mitad de la lista
    mitad = len(v) // 2
    
    if v[mitad] == elem:  # lo hemos encontrado
        return True, mitad
    
    elif elem < v[mitad]:  # el elemento está a la izquierda
        return busca_binaria(v[:mitad], elem)
    
    else:  # el elemento está en la mitad derecha
        encontrado, donde = busca_binaria(v[mitad + 1:], elem)
        return encontrado, mitad + 1 + donde

In [15]:
busca_binaria([1, 3, 7, 9], 3)

(True, 1)

In [16]:
busca_binaria([1, 3, 7, 9], 5)

(False, 2)

## Ejecución de un programa recursivo

Expresar un algoritmo recursivamente permite preocuparse únicamente de la reducción local de un problema a sus subproblemas, sin necesidad de contemplar el cálculo completo. Cuando escribimos $\mathrm{factorial}(0) = 1$ y $\mathrm{factorial}(n) = n \cdot \mathrm{factorial}(n-1)$ (o la función equivalente en Python) es suficiente ver que son identidades válidas para saber que el algoritmo es correcto y que el subproblema de la derecha es más pequeño ($n - 1 < n$) para asegurar que el cálculo termina. No es preciso deducir que se acabarán multiplicando todos los números del $1$ al $n$ y en qué orden para entender que el programa funcionará correctamente. 

Sin embargo, no deja de ser relevante conocer cómo se ejecuta un programa recursivo y sus implicaciones. En realidad, no hay nada novedoso en ello, una llamada recursiva es una llamada normal a función como las que llevamos haciendo desde el primer cuatrimestre. Como hemos observado al ejecutar los programas, una llamada interrumpe la ejecución de la función que se está ejecutando, cuyo estado completo queda pendiente en una estructura conocida como *pila de llamadas*. Esa estructura es una *pila* como las que vimos en la hoja de problemas sobre clases y objetos, es decir, solo permite apilar una nueva llamada sobre las demás, acceder a la información de la llamada actual o desapilarla (como una pila de platos en un fregadero estrecho). Ahí se guardan para cada llamada sus variables y el punto del programa al que habrá que *retornar* cuando la función termine. En más detalle, cuando se llama a una función $f(e_1, ..., e_n)$ se ejecutan aproximadamente los siguientes pasos:
1. se evalúan las expresiones $e_k$ que ocupan sus argumentos,
2. se apila un nuevo contexto en la pila de llamadas con las variables $x_k$ correspondientes a los argumentos (las que aparecen tras el `def`) asignadas al valor de su correspondiente $e_k$. También se guardarán allí las otras variables de la nueva función. Esto implica que cada llamada tiene una copia distinta de sus variables, que aunque tengan el mismo nombre son completamente independientes. Como se ha dicho, se guarda además el punto del programa donde se quedó pendiente la ejecución de la función llamante, para retomarlo en ese punto cuando la función llamada termine.
3. se ejecuta el código de la función $f$ utilizando estas variables locales a su llamada,
4. cuando la llamada a $f$ termina (porque llega al final o al ejecutar un `return`), se desapila su contexto de la pila de llamadas, se recuperan por tanto las variables de la función llamante y se vuelve al punto donde se quedó. En la expresión que incluía la llamada ahora se usa el valor devuelto por `return` (o `None` si no hay `return`).

Veamos como ejemplo la evaluación detallada de `factorial(1)`. En primer lugar, se evalúa el argumento `1` cuyo valor es 1, se apila un contexto para `factorial` con la variable $n=1$ y se empieza a ejecutar la definición de la función `factorial`. En ella se toma la segunda rama ($n \geq 1$), que ha de evaluar `n * factorial(n - 1)`. Por tanto, se hace la llamada recursiva `factorial(n - 1)`, donde primero se evalúa `n - 1` a 0, se apila un nuevo contexto con la variable $n=0$ y se ejecuta la definición de `factorial`. Esta vez se toma la primera rama porque `n == 0` es cierto y se devuelve el valor 1 con `return`. En ese momento, se desapila el contexto de `factorial(0)` y se continúa evaluando la expresión `n * factorial(n - 1)` en `factorial(1)`, que ahora vale `1 * factorial(0)`, luego `1 * 1`, luego 1. Este segundo `return` vuelve a desapilar el contexto de `factorial(1)` y se termina la ejecución devolviendo el valor 1. <bigskip/>

*Observación 2*: la pila de llamadas ocupa memoria y manipularla lleva cierto tiempo, así que las funciones recursivas son en general más lentas que sus equivalentes iterativas (es decir, con bucles). Además, en Python también hay una limitación explícita del número de llamadas anidadas que se pueden hacer, esto es, de la longitud de esa pila de llamadas.

In [17]:
import sys
sys.getrecursionlimit()

3000

In [18]:
fibonacci_mejor(3001)

RecursionError: maximum recursion depth exceeded

In [19]:
fibonacci_iter(3001) % 100000

98001

En ocasiones puede ser necesario extender ese límite para que la llamada recursiva funcione. El límite por defecto en Python es 1.000, pero en IPython (es decir, en Jupyter y en la consola de Spyder) se sube por defecto a 3.000. Esto se puede hacer con la siguiente función.

In [20]:
sys.setrecursionlimit(4000)
fibonacci_mejor(3001)[0] % 100000

98001

## Generalización del problema

La búsqueda binaria del ejemplo anterior es innecesariamente ineficiente, pues las búsquedas recursivas se hacen sobre una copia parcial `v[mitad:]` o `v[:mitad]` de la lista original. Estas copias se pueden evitar generalizando el problema de «buscar `elem` en `v`» (`busca_binaria(v, elem)`) a «buscar `elem` en `v`entre las posiciones `m` y `n`» (`busca_binaria(v, elem, m, n)`).

In [21]:
def busca_binaria2_aux(v: list, elem, m: int, n: int) -> tuple[bool, int]:
    """Busca elem en v entre m y n"""
    
    # Lista vacía
    if n <= m:
        return False, n
    
    # Comprueba el elemento en la mitad de la lista
    mitad = (m + n) // 2
    
    if v[mitad] == elem:
        return True, mitad
    
    elif elem < v[mitad]:
        return busca_binaria2_aux(v, elem, m, mitad)
    
    else:
        return busca_binaria2_aux(v, elem, mitad, n)

def busca_binaria2(v: list, elem) -> tuple[bool, int]:  # llamada inicial
    """Busca elem en v entre m y n"""
    return busca_binaria2_aux(v, elem, 0, len(v))  # fija los valores iniciales

In [22]:
busca_binaria2([1, 3, 7, 9], 3)

(True, 1)

In [23]:
%timeit busca_binaria(list(range(0, 1000)), 501)

14.5 µs ± 347 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
%timeit busca_binaria2(list(range(0, 1000)), 501)

12.4 µs ± 521 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


La generalización de los parámetros de una función recursiva puede resultar conveniente e incluso imprescindible en diversas situaciones.

## [Extra] Memorización para evitar cálculos repetidos

Cuando se hacen múltiples llamadas recursivas, como en la función `fibonacci` del ejemplo 1 bis, es posible que sus subproblemas no sean completamente disjuntos y se repitan cálculos. Veamos que esa repetición puede ser considerable con el siguiente ejemplo.

<noindent/>**Ejemplo 4 (números combinatorios):** el número combinatorio o coeficiente binomial $C^m_n$ representa el número de subconjuntos distintos de tamaño $k$ en un conjunto de tamaño $n$. Sabemos que $C^m_0 = C^m_m = 1$ y que $C^m_n = C^{m-1}_n + C^{m-1}_{n-1}$ en otro caso. Esta recurrencia se puede programar fácilmente como una función recursiva.

In [25]:
veces = {}  # veces que se llama a la función con cada argumento

def comb(m, n):
    """Número combinatorio m sobre n"""
    
    # Cuenta las veces que se llama
    veces[(m, n)] = veces.get((m, n), 0) + 1
    
    if n == 0 or n == m:  # caso base
        return 1
    else:  # caso recursivo
        return comb(m - 1, n) + comb(m - 1, n - 1)

Con ayuda del diccionario `veces`, vamos a contar cuántas veces se llama a la función `comb` con cada posible par de argumentos. 

In [26]:
comb(26, 14)

9657700

En el diccionario ahora podemos ver que muchas llamadas se repiten muchas veces, incluso más de un millón para unos números relativamente pequeños.

In [27]:
{args: cuenta for args, cuenta in veces.items() if cuenta > 1e6}

{(3, 2): 1352078,
 (2, 2): 1352078,
 (2, 1): 2496144,
 (1, 1): 2496144,
 (1, 0): 2496144,
 (3, 1): 1144066,
 (2, 0): 1144066}

Obviamente, eso hace que cálculo se haga extraordinariamente lento.

In [28]:
def comb(m, n):
    """Número combinatorio m sobre n"""  # lo mismo sin usar veces
    
    if n == 0 or n == m:  # caso base
        return 1
    else:  # caso recursivo
        return comb(m - 1, n) + comb(m - 1, n - 1)

In [29]:
%timeit comb(26, 14)

2.02 s ± 27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Es posible evitar cálculos repetidos a costa de la memoria. Igual que hemos usado el diccionario para contar el número de llamadas por cada argumento, podemos utilizarlo para guardar el resultado de la función para cada entrada y así reutilizarlo en futuras llamadas reincidentes sin repetir los cálculos.

In [30]:
resultados = {}

def comb(m, n):
    """Número combinatorio m sobre n"""
    
    if n == 0 or n == m:  # caso base (no hace falta memorizarlo)
        return 1
    else:  # caso recursivo
        if (m, n) in resultados:  # si ya está calculado
            comb_mn = resultados[(m, n)]
        else:
            comb_mn = comb(m - 1, n) + comb(m - 1, n - 1)  # lo calculamos por primera vez
            resultados[(m, n)] = comb_mn  # lo guardamos para la próxima

        return comb_mn

En el caso recursivo (podríamos haberlo hecho también para los casos base) se comprueba si $C^m_n$ ya se había calculado y se hacen las llamadas recursivas solo si no es así. Usando `%timeit` podemos comprobar que el cálculo ahora es mucho más rápido.

In [31]:
%%timeit
resultados.clear()  # borramos el diccionario
comb(26, 14)

79.7 µs ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Hemos usado la variante `%%timeit` (con dos porcentajes) para poder limpiar el diccionario con `resultados.clear()` entre cada dos ejecuciones que realiza este comando para medir los tiempos. En otro caso, tras la primera llamada a `comb(26, 14)`, el resto serían meras consultas al diccionario ya poblado en la anterior ejecución.

In [32]:
%timeit comb(26, 14)  # va aún más rápido, solo consulta el diccionario

219 ns ± 1.28 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Esta ganancia en velocidad se consigue a costa de dedicar memoria a almacenar los resultados de los cálculos intermedios, pero en muchas ocasiones merece la pena.

In [33]:
len(resultados)  # hay 168 entradas en el diccionario

168

La recursión con almacenamiento de los resultados intermedios cuando hay solapamiento forma parte de una técnica algorítmica muy utilizada conocida como *programación dinámica*. Como alternativa a guardar todos esos resultados intermedios, también se consideran otras formas de abordar el problema para evitar que esto sea necesario o reducir la cantidad de información que hay que almacenar, como hicimos con `fibonacci_mejor`.

En Python no es necesario añadir el diccionario manualmente a nuestras funciones si queremos memorizar los resultados, porque cuenta con un decorador `cache`  en el paquete `functools` que hace esa transformación por nosotros.

In [34]:
from functools import cache

Por ejemplo, la función `fibonacci` del ejemplo 1 bis se puede *decorar* con `@cache` para que memorice sus resultados intermedios. Los decoradores (como el `@staticmethod` que vimos en el tema anterior) se escriben sobre la definición de una función para transformarla según interese.

In [35]:
@cache
def fibonacci(n: int) -> int:
    if n == 0 or n == 1:  # caso base
        return n
    else:  # caso inductivo
        return fibonacci(n - 1) + fibonacci(n - 2)

Como hicimos con `comb`, para medir el tiempo de forma justa, hemos de borrar el diccionario entre cada par de ejecuciones. En este caso utilizamos el método `cache_clear` de la función decorada.

In [36]:
%%timeit
fibonacci.cache_clear()  # limpia la cache de la ejecución anterior
fibonacci(30)

7.28 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Esta versión de `fibonacci` es bastante más rápida que la anterior (la misma sin `@cache`), pero no es significativamente más rápida o más lenta que `fibonacci_mejor` y es más lenta que la versión iterativa. La función decorada también tiene un método `cache_info` que informa de cuántas veces ha sido útil el diccionario, su tamaño actual, etc.

In [37]:
fibonacci.cache_info()

CacheInfo(hits=28, misses=31, maxsize=None, currsize=31)

## [Extra] Razonar sobre los algoritmos

El mayor nivel de abstracción de los algoritmos recursivos facilita razonar sobre ellos, especialmente por inducción. Por ejemplo, mirando la definición de la función `fibonacci` del ejemplo 1 bis, es fácil deducir que el número de sumas $s_n$ que se ejecutan al evaluar `fibonacci(n)` es $s_0 = 0$, $s_1 = 0$ y $s_n = 1 + s_{n-1} + s_{n-2}$. A partir de esta recurrencia, podemos probar que el número de sumas que se ejecutan es exponencial.

<noindent/>**Ejemplo 4 (proposición):** *el número de sumas que ejecuta `fibonacci(n)` es mayor o igual que $\varphi^{n - 2}$ para todo $n \geq 2$, donde $\varphi = (1 + \sqrt{5})/2$ es el número áureo.*

En definitiva, hay que probar que $s_n \geq \varphi^{n-2}$ para todo $n \geq 2$. Los casos base son $n = 2$ y $n = 3$ (la propiedad no es cierta para $n$ menor).

* Si $n = 2$, $c_2 = 1$ y $\varphi^{2-2} = \varphi^0 = 1$, luego $1 \geq 1$.
* Si $n = 3$, $c_3 = 2$ y $\varphi^{3-2} = \varphi \geq 1,618$, luego $2 \geq \varphi$.
* Si $n \geq 4$, $$s_n = 1 + s_{n-1} + s_{n-2} \geq s_{n-1} + s_{n-2} \geq_{(HI)} \varphi^{n-3} + \varphi^{n-4} = \varphi^{n-4}(\varphi + 1) = \varphi^{n-4} \cdot \varphi^2 = \varphi^{n-2}$$
donde $\varphi + 1 = \varphi^2$ se puede comprobar fácilmente. Entonces $s_n \geq \varphi^{n-2}$ como queríamos.

La cota es demasiado burda, vale incluso para la sucesión de Fibonacci, pero es suficiente para ver que el número de sumas crece exponencialmente. Por curiosidad, una fórmula exacta es $s_n = ( \varphi^{n+1} - (1 - \varphi)^{n+1} - \sqrt{5}) / \sqrt{5}$.

## Referencias

* §6 «Recursion» del [libro de Guttag](https://ucm.on.worldcat.org/oclc/1347116367) (§4.3 en la [edición de 2013](https://ucm.on.worldcat.org/oclc/1025935018)).